
Copyright

Jelen forráskód a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük
az alábbi szerzőt értesíteni.

2020 (c) Csapó Tamás Gábor (csapot kukac tmit pont bme pont hu),
Gyires-Tóth Bálint, Zainkó Csaba


Links:
    [glove] https://nlp.stanford.edu/projects/glove/
    [glove.6B.50d.txt] http://nlp.stanford.edu/data/glove.6B.zip


In [1]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip glove.6B.50d.txt

--2020-11-04 15:39:25--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  1.92MB/s    in 6m 28s  

2020-11-04 15:45:53 (2.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        


In [2]:
#!wget http://smartlab.tmit.bme.hu/dl/glove.6B.50d.zip
#!unzip glove.6B.50d.zip

In [3]:
import numpy as np
words = set()
vectors = {}
with open('glove.6B.50d.txt', 'r', encoding="utf8", errors='ignore') as f:
  for line in f:
    line = line.strip().split()
    cword = line[0]
    
    try:
      v= np.array(line[1:], dtype=np.float64)
      if v.shape[0] == 50:
          vectors[cword]=v
          words.add(cword)
      else:
          print("Error in line: {}".format(line))
    except:
          print("Error in line: {}".format(line))
print("Number of read words: {}".format(len(words)))


Number of read words: 400000


In [4]:
def find_similarity(word, vector=None):
    if vector is not None:  #Ez a rész az analógiákhoz kell majd, első lépésként nem is kell
       v1=vector            #
    else:                   #
      word=word.lower()
      v1=vectors[word]
    top = 5 
    max= np.ones(top)* -200
    mwords= np.chararray(top,itemsize=20,unicode=True)
    mwords[:]= None
    
    #cos similarity
    u = np.sqrt(np.sum(np.square(v1)))
    
    for w in words:  
        if w in word:
           continue      
        vm=vectors[w]
        uv = np.dot(v1,vm)
        v = np.sqrt(np.sum(np.square(vm)))
        cosine_sim=  uv/(u*v)
    
        
        for i in range(top):
          if cosine_sim > max[i]:
              for j in range(top-1,i,-1):
                max[j]=max[j-1]
                mwords[j]=mwords[j-1]
              #print(w)
              mwords[i]=w
              max[i]=cosine_sim
              break
            
        
    return mwords,max



In [5]:
def find_and_print_similarity(w1):
  print("Find similarity: {}".format(w1))
  top5_words,values=find_similarity(w1)

  for i, w in enumerate(top5_words):
     print("{}. {:6.4f} : {}".format(i,values[i],w))

In [6]:
find_and_print_similarity("dollar")

Find similarity: dollar
0. 0.8264 : euro
1. 0.8199 : currency
2. 0.7883 : dollars
3. 0.7817 : greenback
4. 0.7810 : price


In [7]:
find_and_print_similarity("sea")

Find similarity: sea
0. 0.8812 : ocean
1. 0.8739 : waters
2. 0.8261 : seas
3. 0.7974 : coast
4. 0.7801 : mediterranean


In [8]:
def find_analogy(word1, word2, word3):
    word1=word1.lower()
    word2=word2.lower()
    word3=word3.lower()
    
    v1=vectors[word1]
    v2=vectors[word2]
    v3=vectors[word3]
 
    #cos similarity
    v= v1 - v2 + v3
        
    return find_similarity([word1,word2,word3],v) #word1,word2,word2-at azért adjuk meg, hogy az kizárjuk a kereséből


In [9]:
def find_and_print_analogy(w1,w2,w3):
  print("Find analogy: {} - {} + {} =".format(w1,w2,w3))
  top5_words,values=find_analogy(w1,w2,w3)

  for i, w in enumerate(top5_words):
     print("{}. {:6.4f} : {}".format(i,values[i],w))

In [10]:
find_and_print_analogy('queen', 'woman', 'man')

Find analogy: queen - woman + man =
0. 0.8589 : king
1. 0.8052 : prince
2. 0.7755 : crown
3. 0.7512 : coronation
4. 0.7463 : royal


In [11]:
find_and_print_analogy('Paris', 'France', 'Germany')

Find analogy: Paris - France + Germany =
0. 0.9181 : berlin
1. 0.8184 : frankfurt
2. 0.8121 : munich
3. 0.8101 : vienna
4. 0.7974 : hamburg


In [12]:
find_and_print_analogy('soccer','usa','England')

Find analogy: soccer - usa + England =
0. 0.7775 : premiership
1. 0.7584 : cricket
2. 0.7567 : rugby
3. 0.7370 : football
4. 0.7364 : spell


In [13]:
find_and_print_analogy( 'euro','Europeans', 'americans')

Find analogy: euro - Europeans + americans =
0. 0.8293 : dollar
1. 0.7800 : drop
2. 0.7632 : deficit
3. 0.7622 : us
4. 0.7616 : year


In [14]:
find_and_print_analogy('road','car','ship')

Find analogy: road - car + ship =
0. 0.7876 : harbour
1. 0.7655 : shore
2. 0.7468 : sea
3. 0.7464 : harbor
4. 0.7276 : river


In [15]:
find_and_print_analogy('road','car','boat')

Find analogy: road - car + boat =
0. 0.7612 : shore
1. 0.7449 : river
2. 0.7402 : harbour
3. 0.7215 : shores
4. 0.7212 : inland


In [16]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 50)

In [17]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.50d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)


In [18]:
result = model.most_similar(['sea'], topn=5)
print(result)

[('ocean', 0.8811648488044739), ('waters', 0.8738826513290405), ('seas', 0.8261217474937439), ('coast', 0.7973910570144653), ('mediterranean', 0.7800909876823425)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)
print(result)

[('queen', 0.8523603677749634), ('throne', 0.7664334177970886), ('prince', 0.759214460849762), ('daughter', 0.7473883032798767), ('elizabeth', 0.7460220456123352)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
